# Exercises

1. What does a SavedModel contain? How do you inspect its content?
2. When should you use TF serving? What are its main features? What are some tools you can use to deploy it?
3. How do you deploy a model across multiple TF serving instances?
4. When should you use the gRPC API rather than the REST API to query a model served by TF serving?
5. What are the different ways TFLite reduces a model's size to make it run on a mobile or embedded device?
6. What is quantisation-aware training, & why would you need it?
7. What are model parallelism & data parallelism? Why is the latter generally recommended?
8. When training a model across multiple servers, what distribution strategies can you use? How do you choose which one to use?
9. Train a model (any model you like) & deploy it to TF serving or Google Cloud AI Platform. Write the client code to query it using the REST API or the gRPC API. Update the model & deploy the new version. Your client code will now query the new version. Roll back to the first version.
10. Train any model across multiple GPUs on the same machine using the `MirroredStrategy` (if you do not have access to GPUs, you can use Colaboratory with GPU Runtime & create two virtual GPUs). Train the model again using the `CentralStorageStrategy` & compare the training time.
11. Train a small model on Google Cloud AI Platform, using black box hyperparameter tuning.

---

1. A SavedModel is a directory containing a *.pb* file, which defines the computation graph (represented as a serialised protocol buffer), & a *variables* subdirectory containing the variable values. The variables subdirectory can contain multiple files of variable values, if the model contains a large number of weights. It also contains an *assets* subdirectory, which contains additional data like vocabulary files, class names, or example instances of your model. To inspect its content, you can use the command-line tool `saved_model_cli` to display one or more metagraphs. These metagraphs are computation graphs plus function signature definitions (input, output, names, types, shape).
2. You should use TF serving when you want to deploy multiple TensorFlow models or multiple versions of a model & make them accessible to your applications through either a REST API or gRPC API. TF serving has many features, such as monitoring directories & automatically deploying the latest model version in that directory without having to restart or change your applications. You can manage you gets to see what version of your model; this could be useful for A/B testing where you let a group of users experiment with your new model. To deploy TF serving, you can either install it from the source or install it using a docker image. To deploy a cluster of TF serving docker images, you can use Kubernetes or Google Cloud AI Platform.
3. You configure your TF serving instances so that they all monitor the same directory. Then just export your SavedModel as a subdirectory.
4. You can use the REST API as long as the input & output data aren't too large. However, because of inefficiencies with its serialisation/deserialisation, it results in high latency & bandwidth usage when transferring large numpy arrays. gRPC is more efficient because of this, but its client libraries aren't as widely available as REST.
5. The TFlite library provides several tools to help deploy your models to mobile & embedded devices with three main objectives. One of those objectives is to reduce the model size, to shorten download time & reduce RAM usage. The TFlite model converter can take a SavedModel & compress it to a much lighter format: FlatBuffers. Flatbuffers can be loaded straight to RAM without preprocessing, thus reducing loading time & memory footprint. The model converted also optimises the model by pruning operations that aren't needed to make predictions (training operations), optimising computations, & fusing computations (batch normalisation layers & folder into the previous layer). The model converter can even go as far as to quantise the model weights to 8-bit integers; however, this does reduce the model's accuracy because the converting the quantised weights back to 32-bit floats does give you perfectly identical results. This is a 4-fold reduction though, if you use 32-bit floats. Another way you can reduce the model size is to use smaller bit-widths, for example, using 16-bit floats instead of 32-bit floats.
6. The problem with quantisation is the loss of accuracy when converting back to 32-bit floats. If the accuracy drop is severe, then quantisation-aware training may be necessary. This means adding fake quantisation operations to the model, so that it could learn to ignore the quantisation noise during training, making the weights more robust to quantisation.
7. There are two main approaches to training a single model across multiple devices: model parallelism, where the model is split across devices, & data parallelism, where the model is replicated across every device & each replica is trained on a subset of data. Model parallelism is tricky, because it depends the architecture of your network & how you split it vertically. There can be a lot of cross-device communication going on after the split, & this may completely cancel out the benefits of parallel computation, since cross-device communication is typically slow. Data parallelism is therefore preferred, since it does not have to worry about cross-device communication -- each device has its own replica of the model.
8. You can use `MultiWorkerMirroredStrategy` to perform mirrored data parallelism, where the model is replicated across all available servers & devices & each replica gets a different batch of data at each training iteration to compute its own gradients. The mean of these gradients are then computed & shared across all replicas using a distributed AllReduce implementation (NCCL by default), & all replicas perform the same gradient descent step. This strategy is simple & works well, but its main limitation is that the model must fit in RAM on every replica. The other method is to use `ParameterServerStrategy` to perform asynchronous data parallelism. The model is replicated across all devices on all workers, & the parameters are sharded across all parameter servers. Each worker gets its own training loop, running asynchronously with other workers at each training iteration, each worker gets its own batch of data & fetches the latest version of the model parameters from the parameter servers,t hen it computes the gradients of the loss with regard to these parameters, & it sends them to the parameter servers. Lastly, the parameter servers perform a gradient descent step using these gradients. This method is slower than synchronised data parallelism, & is harder to deploy, but it doesnt require that the models fit into RAM (great to train huge models).